In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# SIGMOD 2022 SystemR Tutorial Code

## System R: Targeted Disinformation for Black-box Models performing End-to-end Training on Structured Data

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(124)

test_size = 0.25
n_target = 10

Adult = Dataset('adult')
target_indices = np.random.choice(Adult.data.index, n_target)
(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta), tr_scaler = Adult.split_dataset(test_size, target_indices)

In [3]:
from model import SurrogateModels
    
model_names = ['nn_tanh_10_2','nn_relu_5_2', 'rf_entropy', 'gb', 'ada', 'log_reg']

s_models = SurrogateModels(model_names)
s_models.train_all(x_tr, y_tr)
s_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])
#s_models.cross_validation(x_tr, y_tr, k=3)

train models..


100%|██████████| 6/6 [00:56<00:00,  9.48s/it]
6it [00:02,  2.92it/s]


,train acc,test acc,target acc
s-nn_tanh_10_2,0.8635,0.8480,1.0
s-nn_relu_5_2,0.8575,0.8529,1.0
s-rf_entropy,0.8526,0.8505,1.0
s-gb,0.8541,0.8544,1.0
s-ada,0.8625,0.8620,1.0
s-log_reg,0.8492,0.8497,1.0


In [4]:
from prob_decision_boundary import PDB

prob_dec = PDB(s_models.models)
x_all = np.concatenate([x_tr, x_te], axis=0)
prob_dec.fit_all(x_all)
sn_te_labels = prob_dec.predict(x_te)

100%|██████████| 6/6 [00:00<00:00, 10.67it/s]


In [5]:
sn_te_labels[sn_te_labels == -1] = 0
te_acc = sum(sn_te_labels==y_te)/len(y_te)

# Candidate Generation
## GAN-based

In [6]:
from gen_disinfos import GANcandidates

adult = Adult.data
column_cat = Adult.column_cat
column_int = Adult.column_int
columns_1hot = Adult.data_1hot.columns

gan_gen = GANcandidates()
gan_gen.fit(adult, column_cat, column_int)

In [7]:
_ = gan_gen.generate()
gan_cand_list = gan_gen.nearest_points(tr_scaler, target_indices, columns_1hot)

In [8]:
from IPython.display import display
import pandas as pd

display(pd.concat([g.iloc[[0]]for g in gan_cand_list],ignore_index=True))

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,30,Private,393267,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,39,United-States
1,48,Private,87800,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
2,31,Private,200488,Bachelors,12,Never-married,Transport-moving,Not-in-family,White,Male,9,0,47,United-States
3,19,Private,259105,5th-6th,3,Never-married,Other-service,Other-relative,White,Male,0,0,47,Mexico
4,39,Private,290382,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,1863,36,United-States
5,29,State-gov,73547,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Male,6,0,45,United-States
6,57,Private,144008,Some-college,9,Separated,Adm-clerical,Not-in-family,White,Female,9,0,39,United-States
7,43,Private,211241,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1822,40,United-States
8,31,Private,97584,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,7,0,47,United-States
9,28,State-gov,224785,Bachelors,10,Never-married,Other-service,Not-in-family,Other,Female,21,0,40,United-States


## WM-based

In [9]:
from gen_disinfos import WMcandidates, agg_disinfo
from tqdm import tqdm
  
adult_1hot = Adult.data_1hot
adult_label = Adult.label

wm_gen = WMcandidates(adult_1hot, adult_label, target_indices)
wm_cand_list = wm_gen.watermarking(tr_scaler, adult.columns, column_cat, column_int)

100%|██████████| 10/10 [04:41<00:00, 28.17s/it]


In [10]:
display(pd.concat([w.iloc[[0]]for w in wm_cand_list],ignore_index=True))

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,23,Private,340691,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,46,United-States
1,57,Private,102241,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
2,35,Private,274574,Bachelors,13,Divorced,Transport-moving,Not-in-family,White,Male,0,0,41,United-States
3,21,Private,521944,5th-6th,3,Never-married,Other-service,Other-relative,White,Male,0,0,40,Mexico
4,31,Private,183680,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,1902,45,United-States
5,25,State-gov,105922,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Male,0,0,40,United-States
6,64,Private,144672,Some-college,10,Separated,Adm-clerical,Not-in-family,White,Female,0,0,38,United-States
7,44,Private,228374,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,1982,40,United-States
8,25,Private,51185,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States
9,28,State-gov,201319,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Other,Female,0,0,41,United-States


In [11]:
x_dis, y_dis = [], []
for ti in range(n_target):
    xt, yt = x_ta[ti], y_ta[ti]
    wm_cand = wm_cand_list[ti]
    gan_cand = gan_cand_list[ti]
    candidates = pd.concat((wm_cand, gan_cand))
    
    x_tmp, y_tmp = agg_disinfo(prob_dec, candidates, tr_scaler, x_tr, y_tr, xt, yt, 
                               columns_1hot, n_disinfo=100)
    x_dis.extend(x_tmp)
    y_dis.extend(y_tmp)

100%|██████████| 6/6 [00:00<00:00, 204.85it/s]


# Insert Disinformation

In [12]:
from model import VictimModels

v_models = VictimModels()
v_models.train_all(x_tr, y_tr)
result_clean = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [18:01<00:00, 56.93s/it]
19it [03:38, 11.49s/it]


In [13]:
x_tr_dis = np.concatenate((x_tr, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tr, y_dis), axis=0).astype(int)

In [14]:
v_models_dis = VictimModels()
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [18:27<00:00, 58.26s/it] 
19it [04:00, 12.66s/it]


In [15]:
from utils import compare_result

compare_result(result_clean, result_dis)

,mean,std
train acc,-0.396316,0.345980
test acc,-0.306842,0.564132
target acc,-12.105263,12.283208


# Membership Inference Attack

In [16]:
from sklearn.model_selection import train_test_split
from model import AttackModels, attack_input

np.random.seed(726)

vi = 1
victim_clean = v_models.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_clean, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [17]:
model_names = ['nn_tanh_5_2','nn_relu_5_2', 'nn_identity', 'tree_gini', 'tree_entropy',
               'rf_gini', 'rf_entropy', 'ada', 'log_reg']

a_models = AttackModels(model_names)
a_models.train_all(x_mia_tr, y_mia_tr)
a_result_clean = a_models.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_clean

train models..


100%|██████████| 9/9 [00:22<00:00,  2.46s/it]
9it [00:00,  9.99it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.5092,0.5033,0.4
a-nn_relu_5_2,0.4993,0.5020,0.5
a-nn_identity,0.5053,0.5146,0.5
a-tree_gini,0.5604,0.5119,0.7
a-tree_entropy,0.5601,0.5151,0.7
a-rf_gini,0.5894,0.5210,0.8
a-rf_entropy,0.5899,0.5266,0.8
a-ada,0.5331,0.5052,0.8
a-log_reg,0.5079,0.5107,0.6


In [18]:
victim_dis = v_models_dis.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_dis, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [19]:
a_models_dis = AttackModels(model_names)
a_models_dis.train_all(x_mia_tr, y_mia_tr)
a_result_dis = a_models_dis.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_dis

train models..


100%|██████████| 9/9 [00:19<00:00,  2.22s/it]
9it [00:00, 10.57it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.5106,0.5082,0.5
a-nn_relu_5_2,0.5002,0.4994,0.5
a-nn_identity,0.5088,0.5065,0.5
a-tree_gini,0.5619,0.5270,0.5
a-tree_entropy,0.5622,0.5277,0.5
a-rf_gini,0.5807,0.5243,0.5
a-rf_entropy,0.5882,0.5286,0.5
a-ada,0.5369,0.5208,0.4
a-log_reg,0.5087,0.5050,0.5


In [20]:
idxs = a_result_clean['target attack acc'] >= 0.5
compare_result(a_result_clean.loc[idxs], a_result_dis.loc[idxs])

,mean,std
train attack acc,0.0275,0.401382
test attack acc,0.4025,0.941333
target attack acc,-18.7500,14.577380
